Given link flows and capacity constraints, this function finds a feasible set of link flows. The inputs to the function are:

f(vector): given (possibly infeasible) link flows

u(vector): link capacity constraints to be met

delta(matrix): link*route incidence matrix

od2route(matrix): K*max_r gives indices of all routes for a given od pair

The output of the function is 

f_feas(vector): feasible link flows for given capacity constraints

In [1]:
debug = True
def find_feasible(f,u,delta,od2route):
    import numpy as np
    delta = np.matrix(delta)
    u = np.array(u)
    
    l = np.shape(f)[0] #number of links in network
    K = np.shape(od2route)[0] #number of od pairs in network
    r = np.shape(delta)[1] #number of routes in network
    d = np.zeros((l,r))
    f_feas = f #vector of feasible flows satisfying capacity constraints
    for k in range(K):
        print('OD pair: %d'%(k+1))
        r_k = od2route[k,:] #vector of indices of all routes bw od pair k
        if isinstance(r_k,list): 
            r_k = r_k[0]
        r_k = r_k[np.invert(np.isnan(r_k))] #take only existing routes
        r_k = r_k.astype(int)
        print('Routes %s'%r_k)
        flag1 = [f_feas>u] 
        sat_f = np.nonzero(flag1)[1] #indices of saturated flows
        d = (delta>0) #binary link route incidence matrix
        d = d*1
        #update the incidence matrix to find saturated flows
    
        # for R in r_k:
        for i in sat_f:
            d[i,:] = 2 * d[i,:]
        #        if d[i,R]>0:
        #            d[i,R] = d[i,R]+1 #matrix indicating routes with saturated links
                    #0: link not in route, 1: unsaturated link, 2: saturated link
        sat = np.where(np.max(d[:,r_k],axis=0)==2)[1] #saturated routes
        unsat = np.where(np.max(d[:,r_k],axis=0)==1)[1] #unsaturated routes
        if np.size(sat)==0:#if no saturated route is present for a given od pair
            print('No saturated route for OD pair: %d'%(k+1))
        
        else:#presence of at least one saturated route
            f_r = 1.0
            for r in sat:#For each saturated route
                if (np.max(d[:,r],axis=0)==2)&(f_r>0):
                    l_r = np.where(d[:,r]>0)[0] #links in saturated route r
                    f_r = max(f_feas[l_r]-u[l_r]) #flow to be rerouted from saturated route r
                    flag = 1
                    while flag: # as long as the route r remains saturated
                        unsat = np.where(np.max(d[:,r_k],axis=0)==1)[1] #unsaturated routes
                        cap = np.zeros(np.size(unsat))
                        c = 0
                        for us in unsat: #compute extra capacity for unsaturated routes
                            l_u = np.where(d[:,us]>0)[0] #links in unsaturated route u
                            cap[c] = min(u[l_u]-f_feas[l_u]) #extra capacity of unsaturated route u
                            c = c+1
                        i_unsat_sort = np.flip(np.argsort(cap),0) #unsaturated routes sorted in decreasing order of capacities
                        cap_sort = cap[i_unsat_sort] 
                        if f_r<=cap_sort[0]: #all flow to be rerouted can be put in one unsat route
                            f_feas[np.where(d[:,r]>0)[0]] = f_feas[np.where(d[:,r]>0)[0]]-f_r #remove f_r from sat route r
                            f_feas[np.where(d[:,unsat[i_unsat_sort[0]]]>0)[0]] = f_feas[np.where(d[:,unsat[i_unsat_sort[0]]]>0)[0]]+f_r #move f_r to route with highest capacity
                            gl = np.where(d[:,r]==2)[0]
                            d[np.where(d[:,r]>0)[0],r] = 1
                            d[gl,np.where(d[gl,:]>0)[1]] = 1
                            flag = 0
                        else: #split flow to be rerouted
                            f_feas[np.where(d[:,r]>0)[0]] = f_feas[np.where(d[:,r]>0)[0]]-cap_sort[0] #remove cap_sort[0] from sat route r
                            f_feas[np.where(d[:,unsat[i_unsat_sort[0]]]>0)[0]] = f_feas[np.where(d[:,unsat[i_unsat_sort[0]]]>0)[0]]+cap_sort[0] #move f_r to route with highest capacity
                            f_r = f_r-cap_sort[0]
                            flag = 1

    print('Capacities:')
    print(u)
    print('Feasible flows:')
    print(f_feas)
    return f_feas

In [2]:
import numpy as np
f = np.matrix([3.5,1,2.5,1,3.5])
f = f.reshape((5,1))
u = np.matrix([10.0,10.0,2.0,10.0,10.0])
u = u.reshape((5,1))
delta = np.matrix([[1,1,0],[0,0,1],[0,1,0],[1,0,0],[0,1,1]])
od2route = np.array([[0,1,2]])

In [3]:
f_feas = find_feasible(f,u,delta,od2route)

OD pair: 1
Routes [0 1 2]
Capacities:
[[10.]
 [10.]
 [ 2.]
 [10.]
 [10.]]
Feasible flows:
[[3. ]
 [1.5]
 [2. ]
 [1. ]
 [3.5]]


In [4]:
import numpy as np
f = np.matrix([4.0,2.0,3.0,1.0,5.0])
f = f.reshape((5,1))
u = np.matrix([10.0,10.0,2.0,10.0,10.0])
u = u.reshape((5,1))
delta = np.matrix([[1,1,0,1,0],[0,0,1,0,1],[0,1,0,1,0],[1,0,0,0,0],[0,1,1,0,0]])
od2route = np.array([[0,1,2],[3,4,np.nan]])
f_feas = find_feasible(f,u,delta,od2route)

OD pair: 1
Routes [0 1 2]
OD pair: 2
Routes [3 4]
No saturated route for OD pair: 2
Capacities:
[[10.]
 [10.]
 [ 2.]
 [10.]
 [10.]]
Feasible flows:
[[4.]
 [2.]
 [2.]
 [2.]
 [4.]]


In [7]:
import Frank_Wolf_solver
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Brae = 'data/braess'

network_name = I210

eps=1e-8
nb_iter = 1000
graph, demand = Frank_Wolf_solver.load_network(network_name)

nb_links = int(np.max(graph[:,0])+1)
c = -1
if network_name == Brae:
    demand[0][2] = 10
    u = [11, 11, 2, 11, 11]
else:
    u = np.zeros(nb_links) + 1e+10
    u[6] = 2000
path_flow_matrix, tt_f, delta, route2od = Frank_Wolf_solver.Frank_Wolf_solver(graph, demand, eps, nb_iter, c) # you can add a capacity

f = delta.T @ path_flow_matrix
delta = delta.toarray().T

od2route = np.array([[i for i in range(delta.shape[1])]])

f_feas = find_feasible(f,u,delta,od2route)

def potential(graph, f, c=-1):
    return np.sum(graph[:,3]*f + 1/2*graph[:,4]*(f**2) + 1/3*graph[:,5]*(f**3) + 1/4*graph[:,6]*(f**4) + 1/5*graph[:,7]*(f**5))

path_flow_matrix, tt_f, delta, route2od = Frank_Wolf_solver.Frank_Wolf_solver(graph, demand, eps, nb_iter, u) # you can add a capacity

f_sol = delta.T @ path_flow_matrix
potential_true = potential(graph, f_sol)
potential_estimate = potential(graph, f_feas)

print()
print("Comparing the solutions:")
print("Objective solution: " + str(potential_true))
print("Objective estimate: " + str(potential_estimate))

print("Flow solution: " + str(f_sol))
print("Flow Feasible flow: " + str(f_feas))


  (0, 1)	1.38
  (0, 6)	1.15
  (1, 2)	1.38
  (1, 7)	1.14
  (2, 3)	1.68
  (2, 8)	1.13
  (3, 4)	1.26
  (3, 9)	1.05
  (4, 5)	2.25
  (4, 10)	1.04
  (5, 11)	1.09
  (6, 0)	1.15
  (6, 7)	0.48
  (6, 12)	0.36
  (7, 1)	1.14
  (7, 8)	0.48
  (7, 13)	0.39
  (8, 2)	1.13
  (8, 9)	0.58
  (8, 14)	0.38
  (9, 3)	1.05
  (9, 10)	0.43
  (9, 15)	0.48
  (10, 4)	1.04
  (10, 11)	0.77
  (10, 16)	0.5
  (11, 5)	1.09
  (11, 17)	0.47
  (11, 19)	0.22
  (12, 6)	0.36
  (12, 13)	1.38
  (13, 7)	0.39
  (13, 14)	1.38
  (14, 8)	0.38
  (14, 15)	1.68
  (15, 9)	0.48
  (15, 16)	1.25
  (16, 10)	0.5
  (16, 17)	2.22
  (17, 11)	0.47
  (18, 6)	0.29
[    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0. 25000.     0.     0. 25000.     0.     0. 25000.     0.
     0. 25000.     0.     0. 25000.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0. 25000.
 25000.]
Test of initialization_FW
[    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  

In [8]:
print("Comparing the solutions:")
print("Objective solution: " + str(potential_true))
print("Objective estimate: " + str(potential_estimate))

print("Flow solution: " + str(f_sol))
print("Flow Feasible flow: " + str(f_feas))

Comparing the solutions:
Objective solution: 1.30310704108885e+19
Objective estimate: 1.2245090873910006e+19
Flow solution: [ 2584.23594078     0.          2937.95358835   119.21355823
  3050.58518695     0.          5231.29113505     0.
  2811.29286839  2419.99826666  2811.29286839  2584.23594078
 19577.99952146  2837.76453776   472.9312058  19124.67834845
   407.11507916   112.63159861 19031.71858296    87.142974
  2180.7059481  17142.86571008   100.18725414     0.
 19356.95698312   312.55043204     0.             0.
     0.          2837.76453776   307.51155372  2937.3680632
   106.81480711  2917.69623009   392.04032935  2625.84315487
   106.64343842  2831.7501485   2831.7501485  25000.
 25000.        ]
Flow Feasible flow: [ 1658.85822662     0.          2011.27540856     0.
  2007.22662041    78.82078746  2000.            91.41567024
  1770.37882654   303.46952717  1770.37882654  1658.85822662
 20503.50251338  2837.63926      352.41718193 19125.13862063
  1107.11265808    74.771999